In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import train_test_split

from collections import Counter

In [35]:
import warnings
warnings.filterwarnings("ignore")

In [38]:
with open('/home/jovyan/work/item_coldstart_dataset.pkl', 'rb') as f:
    df = pickle.load(f)

In [156]:
df.head()

,actor,age_access_type,country,average_rating,duration,availability,type,name,release_year,genre,director,target,subscription_only,uid
0,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 21c7a45...",12,[usa],6.21,6420000,"[DTO, RENT, SUBSCRIPTION]",MOVIE,Звёздный путь 5: Последний рубеж,1989,"[Sci-Fi, Action, Thriller, Adventure]",[cff3362f-ad91-498d-80b3-ea3ab8ec65cf],8.508657e-07,False,e785baa6-f175-42b4-9e16-4319ac7991d5
1,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 21c7a45...",16,[usa],7.2,6780000,"[DTO, RENT, SUBSCRIPTION]",MOVIE,Звёздный путь 6: Неоткрытая страна,1991,"[Sci-Fi, Action, Thriller, Adventure]",[bab6b7f4-4506-458f-9091-d567505674f2],1.559920e-06,False,4593737e-de9c-40df-97db-fb3cf85a08ef
2,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, bcad17e...",18,[usa],4.0,6000000,"[DTO, RENT, SUBSCRIPTION]",MOVIE,В поисках древнего артефакта,2019,[Horror],[895f5774-964a-4c5a-ae22-d697d3e3e620],3.578051e-06,False,11ba66db-e941-4c3a-8da6-d8900e56f8c7
3,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 30dd19c...",18,[usa],6.99,3780000,[SUBSCRIPTION],MOVIE,Прожарка Чарли Шина,2011,"[Comedy, Documentary]",[1de22aff-430e-4af9-bf1d-159dbf8e9269],3.544765e-04,True,3f30a2ef-53b7-40e3-954f-1bdfc38a6d17
4,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, a805fdd...",18,[usa],6.0,3780000,[SUBSCRIPTION],MOVIE,Прожарка Уильяма Шэтнера,2006,[Comedy],[1de22aff-430e-4af9-bf1d-159dbf8e9269],6.394210e-05,True,cdfa700f-122d-41e5-b8dc-9c6813bab6d2


In [40]:
df.columns

Index(['actor', 'age_access_type', 'country', 'average_rating', 'duration',
       'availability', 'type', 'name', 'release_year', 'genre', 'director',
       'target', 'subscription_only', 'uid'],
      dtype='object')

In [157]:
df.shape

(11917, 14)

In [48]:
df.drop([2408, 2446], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

# Step for model 3_2

In [52]:
df.head(1)

,actor,age_access_type,country,average_rating,duration,availability,type,name,release_year,genre,director,target,subscription_only,uid
0,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 21c7a45...",12,[usa],6.21,6420000,"[DTO, RENT, SUBSCRIPTION]",MOVIE,Звёздный путь 5: Последний рубеж,1989,"[Sci-Fi, Action, Thriller, Adventure]",[cff3362f-ad91-498d-80b3-ea3ab8ec65cf],8.508657e-07,False,e785baa6-f175-42b4-9e16-4319ac7991d5


In [54]:
actors = []

for values in df['actor']:
    actors.extend(values)
    
actors_count = Counter(actors)

In [62]:
len(actors_count)

34027

In [69]:
actors_count_1 = [actor for actor, c in actors_count.items() if c < 2]

In [70]:
len(actors_count_1)

23878

In [15]:
from gensim.models import Word2Vec
from random import choices

mltpl_cat_features = ['actor', 'country', 'genre', 'director']
cat_features = ['availability']
cat_features_raw = ['type', 'subscription_only']
num_features = ['age_access_type', 'average_rating', 'duration', 'release_year']
skewed_num_features = ['duration']
text_features = ['name']
mltpl_cat_n_top = {'actor': 5, 'country': 1, 'genre': 3, 'director': 1}


class Cat2Vec:
    def __init__(self, n_cat2vec_feature,
                    n_cat2vec_window,
                    min_count=1):
        self.n_cat2vec_feature = n_cat2vec_feature
        self.n_cat2vec_window = n_cat2vec_window
        self.min_count = min_count
    
    def __create_tokens(self, col_name, values):
        return [f'{col_name} {value}' for value in values]
    
    def __gen_cat2vec_sentences(self, data):
        items = []

        for i, row in data.iterrows():
            item_values = []
            for col, values in zip(row.index, row.values):
                item_values.extend(self.__create_tokens(col, values))
            # shuffle(item_values)
            items.append(item_values)
        return items
    
    def fit(self, cat_features):
        X_w2v = self.__gen_cat2vec_sentences(cat_features)
        self.model = Word2Vec(X_w2v, vector_size=self.n_cat2vec_feature, 
                         window=self.n_cat2vec_window, epochs=5, min_count=self.min_count)
        
    def transform(self, cat_features):
        encoded_features = pd.DataFrame()
        for col in cat_features.columns:
            tokens = [self.__create_tokens(col, row) for row in cat_features[col]]
            col_embeds = []
            for row in tokens:
                avg_embeds = np.mean([self.model.wv[token] for token in row if token in self.model.wv.key_to_index], 
                                          axis=0)
                if isinstance(avg_embeds, np.float64):
                    avg_embeds = [np.nan] * self.n_cat2vec_feature
                    
                col_embeds.append(avg_embeds)
            col_embeds = pd.DataFrame(col_embeds, columns=[f'{col}_{i}' for i in range(self.n_cat2vec_feature)])
            encoded_features = pd.concat([encoded_features, col_embeds], axis=1)

        return encoded_features

    
class Preprocesser:
    def __init__(self, 
                    mulpiple_cat_features: list,
                    skewed_num_features: list,
                    text_features: list,
                    cat_features: list,
                    mltpl_cat_n_top: dict,
                    n_cat2vec_feature: int, 
                    n_cat2vec_window: int,
                    min_count=1):
        self.mulpiple_cat_features = mulpiple_cat_features
        self.skewed_num_features = skewed_num_features
        self.text_features = text_features
        self.mltpl_cat_n_top = mltpl_cat_n_top
        self.cat_features = cat_features
        
        self.cat2vec = Cat2Vec(n_cat2vec_feature, n_cat2vec_window,
                               min_count)
        
    def __get_top_n_mltpl_cat_features(self, df_pr, col):
        all_cats = []
        for element_values in df_pr[col]:
            all_cats.extend(element_values)
        return Counter(all_cats).most_common(self.mltpl_cat_n_top[col])
            
    def __preprocess_mulpiple_cat_features(self, df_pr):
        features = self.cat2vec.transform(df_pr[self.mulpiple_cat_features])
        # print('features')
        # print(features.isnull().sum())
        # print('df_pr')
        df_pr[features.columns] = features
        # print(df_pr.isnull().sum())
        df_pr.drop(self.mulpiple_cat_features, axis=1, inplace=True)
        
        return df_pr
    
    def __preprocess_skewed_num_features(self, df_pr):
        for col in self.skewed_num_features:
            df_pr[col] = df_pr[col].apply(lambda x: np.log1p(x))
        return df_pr
    
    def __preprocess_text_features(self, df_pr):
        for col in self.text_features:
            df_pr[[f'{col}_{i}' for i in range(768)]] = self.embed_model.encode(df_pr[col].values)
            df_pr.drop(col, axis=1, inplace=True)
        return df_pr
    
    def __get_unique_values(self, feature):
        unique_values = []
        for element in feature.dropna():
            unique_values.extend(element)
        return set(unique_values)
    
    def __preprocess_cat_features(self, df_pr):
        for col in self.cat_features:
            unique_values = self.__get_unique_values(df_pr[col])
            cats_df = pd.DataFrame(np.full((len(df_pr), len(unique_values)),
                                            0),
                                   columns=list(unique_values))
            
            for i, element in enumerate(df_pr[col]):
                if isinstance(element, float):
                    cats_df.loc[i, :] = 'Na'
                    continue
                for value in element:
                    cats_df.loc[i, value] = '1'
            df_pr.drop(col, axis=1, inplace=True)
        df_pr[cats_df.columns] = cats_df
        return df_pr  
            
    def fit(self, 
            df: pd.DataFrame):
        self.cat2vec.fit(df[self.mulpiple_cat_features])
        
    def preprocess(self,
                   df: pd.DataFrame):
        df_pr = df.copy()
        df_pr = self.__preprocess_mulpiple_cat_features(df_pr)
        df_pr = self.__preprocess_skewed_num_features(df_pr)
        df_pr = self.__preprocess_cat_features(df_pr)
        
        return df_pr
        
        
class TargetTransformer:
    def __init__(self, denominator=1):
        self.scale = denominator
        
    def inverse_transform(self, feature):
        return feature.apply(lambda x: np.expm1(x))
    
    def transform(self, feature):
        return feature.apply(lambda x: np.log1p(x))

# min_count=2

In [174]:
mltpl_cat_features = ['actor', 'country', 'genre', 'director']
cat_features = ['availability']
num_features = ['age_access_type', 'average_rating', 'duration']
skewed_num_features = ['duration']
text_features = ['name']
mltpl_cat_n_top = {'actor': 5, 'country': 1, 'genre': 3, 'director': 1}

cat_cols = ['actor', 'genre', 'director']
n_cat2vec_feature  = len(cat_cols) * 2 # define the cat2vecs dimentions
n_cat2vec_window   = len(cat_cols) * 2

In [175]:
preprocesser = Preprocesser(mltpl_cat_features,
            skewed_num_features,
            text_features,
            cat_features,
            mltpl_cat_n_top,
            n_cat2vec_feature, 
            n_cat2vec_window,
            min_count=2)

target_transformer = TargetTransformer()

In [176]:
df_modern = df[df['release_year'] >= 2000]
df_short = df_modern[df_modern['target'] < 0.02]
df_short.reset_index(drop=True, inplace=True)
preprocesser.fit(df_short)
df_pr = preprocesser.preprocess(df_short)

df_pr.drop('FVOD', axis=1, inplace=True)
df_pr['release_year'] = df_pr['release_year'].astype(str)
df_pr['average_rating'] = df_pr['average_rating'].astype(float)
df_pr['age_access_type'] = df_pr['age_access_type'].astype(int)
df_pr['subscription_only'] = df_pr['subscription_only'].apply(int).astype(str)

In [177]:
df_pr.isnull().sum()

age_access_type         0
average_rating        525
duration                0
type                    0
name                    0
release_year            0
target                  0
subscription_only       0
uid                     0
actor_0               752
actor_1               752
actor_2               752
actor_3               752
actor_4               752
actor_5               752
country_0               2
country_1               2
country_2               2
country_3               2
country_4               2
country_5               2
genre_0                 0
genre_1                 0
genre_2                 0
genre_3                 0
genre_4                 0
genre_5                 0
director_0           3572
director_1           3572
director_2           3572
director_3           3572
director_4           3572
director_5           3572
SUBSCRIPTION            0
DTO                     0
RENT                    0
AVOD                    0
dtype: int64

In [178]:
cat_features = ['type', 'subscription_only', 
                'SUBSCRIPTION', 'AVOD', 'DTO',
                'RENT', 'release_year']
num_features = ['age_access_type', 'average_rating', 'duration']

df_pr[cat_features] = df_pr[cat_features].replace(np.nan, 'Na')

In [179]:
MULTIPLIER = 100
X, y = df_pr.drop(['target', 'name', 'uid'], axis=1), df_pr['target']  * MULTIPLIER
y = target_transformer.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2, random_state=42, shuffle=True)

In [180]:
with open('dataset_2.pkl', 'wb') as f:
    pickle.dump(df_pr, f)

In [181]:
df_pr

,age_access_type,average_rating,duration,type,name,release_year,target,subscription_only,uid,actor_0,...,director_0,director_1,director_2,director_3,director_4,director_5,SUBSCRIPTION,DTO,RENT,AVOD
0,18,4.00,15.607270,MOVIE,В поисках древнего артефакта,2019,0.000004,0,11ba66db-e941-4c3a-8da6-d8900e56f8c7,-0.048132,...,0.146553,0.086444,0.181065,-0.175414,0.169276,0.104724,1,1,1,0
1,18,6.99,15.145235,MOVIE,Прожарка Чарли Шина,2011,0.000354,1,3f30a2ef-53b7-40e3-954f-1bdfc38a6d17,-0.063862,...,-0.096604,0.208678,0.257821,-0.084255,0.145304,0.262974,1,0,0,0
2,18,6.00,15.145235,MOVIE,Прожарка Уильяма Шэтнера,2006,0.000064,1,cdfa700f-122d-41e5-b8dc-9c6813bab6d2,-0.094405,...,-0.096604,0.208678,0.257821,-0.084255,0.145304,0.262974,1,0,0,0
3,0,6.21,15.396549,MOVIE,Большое путешествие,2006,0.000003,1,d8b55146-dee5-4498-957c-5614414b48fb,-0.079006,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
4,6,6.40,15.523889,MOVIE,Пламенное сердце,2022,0.009705,1,ba6bec1a-3aa9-48c7-aa65-908c21627a12,-0.023747,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9478,16,7.60,18.121735,SERIAL,"Учёный, гуляющий по ночам",2015,0.000085,0,f4b8a993-5d91-41f4-b5fb-3769a73d4b53,-0.071564,...,0.119466,0.063323,0.123950,-0.150059,0.144995,0.249488,1,1,0,0
9479,18,NaN,17.184185,SERIAL,Милые обманщицы: Первородный грех,2022,0.000696,1,673a98a8-5d12-4a3d-9c50-d1d196a8fd96,-0.052503,...,0.116897,-0.096789,0.075730,0.020192,-0.100348,0.097352,1,0,0,0
9480,16,7.80,17.627492,SERIAL,Пропавшие: Другая сторона,2020,0.000652,0,37305c51-cb1e-4ff2-b5ad-3425ed73b17a,-0.076522,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,1
9481,16,NaN,16.358686,SERIAL,Тиндер-80,2020,0.001266,1,249b3db3-8dbd-4a18-bd4b-5d0cc08878c4,-0.014324,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0


# min_count=1

In [166]:
preprocesser = Preprocesser(mltpl_cat_features,
            skewed_num_features,
            text_features,
            cat_features,
            mltpl_cat_n_top,
            n_cat2vec_feature, 
            n_cat2vec_window,
            min_count=1)

target_transformer = TargetTransformer()

In [168]:
df_modern = df[df['release_year'] >= 2000]
df_short = df_modern[df_modern['target'] < 0.02]
df_short.reset_index(drop=True, inplace=True)
preprocesser.fit(df_short)
df_pr = preprocesser.preprocess(df_short)

df_pr.drop('FVOD', axis=1, inplace=True)
df_pr['release_year'] = df_pr['release_year'].astype(str)
df_pr['average_rating'] = df_pr['average_rating'].astype(float)
df_pr['age_access_type'] = df_pr['age_access_type'].astype(int)
df_pr['subscription_only'] = df_pr['subscription_only'].apply(int).astype(str)

In [169]:
df_pr

,age_access_type,average_rating,duration,type,name,release_year,target,subscription_only,uid,actor_0,...,director_0,director_1,director_2,director_3,director_4,director_5,SUBSCRIPTION,DTO,RENT,AVOD
0,18,4.00,15.607270,MOVIE,В поисках древнего артефакта,2019,0.000004,0,11ba66db-e941-4c3a-8da6-d8900e56f8c7,-0.030709,...,-0.005546,0.034927,0.179571,0.139724,0.066432,0.127680,1,1,1,0
1,18,6.99,15.145235,MOVIE,Прожарка Чарли Шина,2011,0.000354,1,3f30a2ef-53b7-40e3-954f-1bdfc38a6d17,0.033636,...,-0.139757,-0.044668,0.109483,0.156753,-0.002948,0.036343,1,0,0,0
2,18,6.00,15.145235,MOVIE,Прожарка Уильяма Шэтнера,2006,0.000064,1,cdfa700f-122d-41e5-b8dc-9c6813bab6d2,0.004407,...,-0.139757,-0.044668,0.109483,0.156753,-0.002948,0.036343,1,0,0,0
3,0,6.21,15.396549,MOVIE,Большое путешествие,2006,0.000003,1,d8b55146-dee5-4498-957c-5614414b48fb,-0.028783,...,-0.058414,-0.088417,0.085593,0.010822,-0.128328,0.077235,1,0,0,0
4,6,6.40,15.523889,MOVIE,Пламенное сердце,2022,0.009705,1,ba6bec1a-3aa9-48c7-aa65-908c21627a12,-0.034545,...,-0.063541,0.019863,0.107477,0.056392,0.063167,-0.011712,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9478,16,7.60,18.121735,SERIAL,"Учёный, гуляющий по ночам",2015,0.000085,0,f4b8a993-5d91-41f4-b5fb-3769a73d4b53,-0.017683,...,-0.057429,-0.090846,-0.073086,0.031009,0.168534,0.051915,1,1,0,0
9479,18,NaN,17.184185,SERIAL,Милые обманщицы: Первородный грех,2022,0.000696,1,673a98a8-5d12-4a3d-9c50-d1d196a8fd96,-0.029495,...,0.060532,-0.078773,-0.030466,-0.036163,0.034507,0.017278,1,0,0,0
9480,16,7.80,17.627492,SERIAL,Пропавшие: Другая сторона,2020,0.000652,0,37305c51-cb1e-4ff2-b5ad-3425ed73b17a,0.009376,...,-0.043079,0.042460,0.052136,-0.155240,0.118663,0.167637,1,0,0,1
9481,16,NaN,16.358686,SERIAL,Тиндер-80,2020,0.001266,1,249b3db3-8dbd-4a18-bd4b-5d0cc08878c4,-0.001560,...,0.107688,0.112130,0.068875,-0.118909,-0.016524,-0.025074,1,0,0,0


In [170]:
df_pr.isnull().sum()

age_access_type        0
average_rating       525
duration               0
type                   0
name                   0
release_year           0
target                 0
subscription_only      0
uid                    0
actor_0                0
actor_1                0
actor_2                0
actor_3                0
actor_4                0
actor_5                0
country_0              0
country_1              0
country_2              0
country_3              0
country_4              0
country_5              0
genre_0                0
genre_1                0
genre_2                0
genre_3                0
genre_4                0
genre_5                0
director_0             0
director_1             0
director_2             0
director_3             0
director_4             0
director_5             0
SUBSCRIPTION           0
DTO                    0
RENT                   0
AVOD                   0
dtype: int64

In [172]:
cat_features = ['type', 'subscription_only', 
                'SUBSCRIPTION', 'AVOD', 'DTO',
                'RENT', 'release_year']

df_pr[cat_features] = df_pr[cat_features].replace(np.nan, 'Na')

In [28]:
MULTIPLIER = 100
X, y = df_pr.drop(['target', 'name', 'uid'], axis=1), df_pr['target']  * MULTIPLIER
y = target_transformer.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2, random_state=42, shuffle=True)

In [173]:
with open('dataset_2_1.pkl', 'wb') as f:
    pickle.dump(df_pr, f)

# cat2vwc kp

In [156]:
mulpiple_cat_features = ['country_x', 'ACTOR', 'COMPOSER', 'DIRECTOR',
           'EDITOR', 'OPERATOR', 'PRODUCER', 'WRITER']

cat_feature_tfidf = 'genre_x'

class Cat2Vec:
    def __init__(self, n_cat2vec_feature,
                    n_cat2vec_window):
        self.n_cat2vec_feature = n_cat2vec_feature
        self.n_cat2vec_window = n_cat2vec_window
    
    def __create_tokens(self, col_name, values):
        return [f'{col_name} {value}' for value in values]
    
    def __gen_cat2vec_sentences(self, data):
        items = []

        for i, row in data.iterrows():
            item_values = []
            for col, values in zip(row.index, row.values):
                item_values.extend(self.__create_tokens(col, values))
            shuffle(item_values)
            items.append(item_values)
        return items
    
    def fit(self, cat_features):
        X_w2v = self.__gen_cat2vec_sentences(cat_features)
        self.model = Word2Vec(X_w2v, vector_size=self.n_cat2vec_feature, 
                         window=self.n_cat2vec_window, epochs=100, min_count=2)
        
    def transform(self, cat_features):
        encoded_features = pd.DataFrame()
        for col in cat_features.columns:
            tokens = [self.__create_tokens(col, row) if not isinstance(row, float) else [np.nan] for row in cat_features[col]]
            col_embeds = []
            for row in tokens:
                avg_embeds = np.mean([self.model.wv[token] for token in row if token in self.model.wv.key_to_index], 
                                          axis=0)
                if isinstance(avg_embeds, np.float64):
                    avg_embeds = [np.nan] * self.n_cat2vec_feature
                    
                col_embeds.append(avg_embeds)
            col_embeds = pd.DataFrame(col_embeds, columns=[f'{col}_{i}' for i in range(self.n_cat2vec_feature)])
            encoded_features = pd.concat([encoded_features, col_embeds], axis=1)

        return encoded_features
        
        
class Preprocesser:
    def __init__(self, 
                    mulpiple_cat_features: list,
                    skewed_num_features: list,
                    text_features: list,
                    cat_features: list,
                    mltpl_cat_n_top: dict,
                    cat_feature_tfidf: str):
        self.mulpiple_cat_features = mulpiple_cat_features
        self.skewed_num_features = skewed_num_features
        self.text_features = text_features
        self.mltpl_cat_n_top = mltpl_cat_n_top
        self.cat_features = cat_features
        self.cat_feature_tfidf = cat_feature_tfidf
        
        self.cat2vec = None
        
        self.tfidf = TfidfVectorizer(sublinear_tf=True, min_df=50,
                                     ngram_range=(1, 1),
                                     tokenizer=lambda x: x.split(' '))
        
        
    def __encode_rare(self, df):
        for col in self.mulpiple_cat_features:
            all_occur = []

            for val in df[col]:
                if not isinstance(val, float):
                    all_occur.extend(val)

            all_occur_count = pd.Series(all_occur).value_counts()
            rare_items = all_occur_count[all_occur_count == 1].index

            new_values = []

            for val in df[col]:
                if not isinstance(val, float):
                    new_values.append(list(set([item if item not in rare_items else 'other' for item in val])))
                else:
                    new_values.append(np.nan)


            df[col] = new_values
        return df
        
    def __get_top_n_mltpl_cat_features(self, df_pr, col):
        all_cats = []
        for element_values in df_pr[col]:
            all_cats.extend(element_values)
        return Counter(all_cats).most_common(self.mltpl_cat_n_top[col])
            
    def __preprocess_mulpiple_cat_features(self, df_pr):
        features = self.cat2vec.transform(df_pr[self.mulpiple_cat_features])
        df_pr[features.columns] = features
        df_pr.drop(self.mulpiple_cat_features, axis=1, inplace=True)
        
        return df_pr
    
    def __preprocess_skewed_num_features(self, df_pr):
        for col in self.skewed_num_features:
            df_pr[col] = df_pr[col].apply(lambda x: np.log1p(x))
        return df_pr
    
    def __preprocess_text_features(self, df_pr):
        for col in self.text_features:
            df_pr[[f'{col}_{i}' for i in range(768)]] = self.embed_model.encode(df_pr[col].values)
            df_pr.drop(col, axis=1, inplace=True)
        return df_pr
    
    def __get_unique_values(self, feature):
        unique_values = []
        for element in feature.dropna():
            unique_values.extend(element)
        return set(unique_values)
    
    def __preprocess_cat_features(self, df_pr):
        for col in self.cat_features:
            unique_values = self.__get_unique_values(df_pr[col])
            cats_df = pd.DataFrame(np.full((len(df_pr), len(unique_values)),
                                            0),
                                   columns=list(unique_values))
            
            for i, element in enumerate(df_pr[col]):
                if isinstance(element, float):
                    cats_df.loc[i, :] = 'Na'
                    continue
                for value in element:
                    cats_df.loc[i, value] = '1'
            df_pr.drop(col, axis=1, inplace=True)
        df_pr[cats_df.columns] = cats_df
        return df_pr  
    
    def __preprocess_cat_features_tfidf(self, df_pr):
        features = self.tfidf.transform(df_pr[self.cat_feature_tfidf].apply(lambda x : ' '.join(x))).toarray()
        df_pr[self.tfidf.get_feature_names()] = features

        return df_pr.drop(self.cat_feature_tfidf, axis=1)
  
    def fit(self, df):
        self.tfidf.fit(df[self.cat_feature_tfidf].apply(lambda x : ' '.join(x)))
        
    def preprocess(self,
                   df: pd.DataFrame):
        df_pr = df.copy()
        df_pr = self.__encode_rare(df_pr)
        df_pr = self.__preprocess_mulpiple_cat_features(df_pr)
        df_pr = self.__preprocess_cat_features_tfidf(df_pr)
        
        return df_pr
    
with open('/home/jovyan/work/cat2vec_kp.pkl', 'rb') as f:
    cat2vec = pickle.load(f)
    
preprocesser = Preprocesser(mulpiple_cat_features=mulpiple_cat_features,
                    skewed_num_features=[],
                    text_features=[],
                    cat_features=[],
                    mltpl_cat_n_top=[],
                    cat_feature_tfidf=cat_feature_tfidf
                    )

preprocesser.cat2vec = cat2vec

In [157]:
with open('/home/jovyan/work/cold_start_models/item_coldstart_dataset-avg.pkl', 'rb') as f:
    df_avg = pickle.load(f)
    
    
with open('/home/jovyan/work/cold_start_models/item_coldstart_dataset_kp.pkl', 'rb') as f:
    df_kp = pickle.load(f)

In [158]:
df = df_kp.merge(df_avg, on='uid', how='inner')

In [159]:
df.columns

Index(['age_access_type_x', 'country_x', 'duration_x', 'availability_x',
       'type_x', 'name_x', 'release_year_x', 'genre_x', 'user_count_3months',
       'subscription_only_x', 'uid', 'ACTOR', 'COMPOSER', 'DESIGN', 'DIRECTOR',
       'EDITOR', 'OPERATOR', 'PRODUCER', 'WRITER', 'BUDGET', 'MARKETING',
       'RUS', 'USA', 'WORLD', 'actor', 'age_access_type_y', 'country_y',
       'average_rating', 'duration_y', 'availability_y', 'type_y', 'name_y',
       'release_year_y', 'genre_y', 'director', 'subscription_only_y',
       'target'],
      dtype='object')

In [160]:
features = ['name_x', 'age_access_type_x', 'country_x', 'duration_x', 'type_x',
            'release_year_x', 'genre_x', 'ACTOR', 'COMPOSER', 'DIRECTOR',
           'EDITOR', 'OPERATOR', 'PRODUCER', 'WRITER', 'BUDGET', 'MARKETING',
           'RUS', 'WORLD', 'target', 'uid']

df = df[features]

In [161]:
df_short.columns

Index(['name_x', 'age_access_type_x', 'country_x', 'duration_x', 'type_x',
       'release_year_x', 'genre_x', 'ACTOR', 'COMPOSER', 'DIRECTOR', 'EDITOR',
       'OPERATOR', 'PRODUCER', 'WRITER', 'BUDGET', 'MARKETING', 'RUS', 'WORLD',
       'target', 'uid'],
      dtype='object')

In [162]:
df_short.reset_index(drop=True, inplace=True)
preprocesser.fit(df)
df_pr = preprocesser.preprocess(df)

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/o

In [163]:
df_pr.shape

(10017, 225)

In [164]:
df_pr

,name_x,age_access_type_x,duration_x,type_x,release_year_x,BUDGET,MARKETING,RUS,WORLD,target,...,forkids,history,horror,humor,melodrama,music,sci-fi,sport,thriller,war_movies
0,Звёздный путь 5: Последний рубеж,12,6420000.0,MOVIE,1989.0,27800000.0,NaN,NaN,52210049.0,7.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.564117,0.0,0.436822,0.0
1,Звёздный путь 6: Неоткрытая страна,16,6780000.0,MOVIE,1991.0,30000000.0,NaN,NaN,96888996.0,7.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.564117,0.0,0.436822,0.0
2,В поисках древнего артефакта,18,6000000.0,MOVIE,2019.0,NaN,NaN,NaN,NaN,32.0,...,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
3,Прожарка Чарли Шина,18,3780000.0,MOVIE,2011.0,NaN,NaN,NaN,NaN,122.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
4,Прожарка Уильяма Шэтнера,18,3780000.0,MOVIE,2006.0,NaN,NaN,NaN,NaN,25.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10012,Эрнест и Селестина,0,36300000.0,SERIAL,2017.0,NaN,NaN,NaN,NaN,729.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
10013,"Мульт ""Кухня""",16,29040000.0,SERIAL,2017.0,NaN,NaN,NaN,NaN,17.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
10014,Без памяти,12,12240000.0,SERIAL,2021.0,NaN,NaN,NaN,NaN,7164.0,...,0.0,0.0,0.0,0.0,0.804481,0.0,0.000000,0.0,0.000000,0.0
10015,Всплеск любви,16,8400000.0,SERIAL,2015.0,NaN,NaN,NaN,NaN,990.0,...,0.0,0.0,0.0,0.0,0.634899,0.0,0.000000,0.0,0.000000,0.0


In [165]:
list(df_pr.columns)

['name_x',
 'age_access_type_x',
 'duration_x',
 'type_x',
 'release_year_x',
 'BUDGET',
 'MARKETING',
 'RUS',
 'WORLD',
 'target',
 'uid',
 'country_x_0',
 'country_x_1',
 'country_x_2',
 'country_x_3',
 'country_x_4',
 'country_x_5',
 'country_x_6',
 'country_x_7',
 'country_x_8',
 'country_x_9',
 'country_x_10',
 'country_x_11',
 'country_x_12',
 'country_x_13',
 'country_x_14',
 'country_x_15',
 'country_x_16',
 'country_x_17',
 'country_x_18',
 'country_x_19',
 'country_x_20',
 'country_x_21',
 'country_x_22',
 'country_x_23',
 'ACTOR_0',
 'ACTOR_1',
 'ACTOR_2',
 'ACTOR_3',
 'ACTOR_4',
 'ACTOR_5',
 'ACTOR_6',
 'ACTOR_7',
 'ACTOR_8',
 'ACTOR_9',
 'ACTOR_10',
 'ACTOR_11',
 'ACTOR_12',
 'ACTOR_13',
 'ACTOR_14',
 'ACTOR_15',
 'ACTOR_16',
 'ACTOR_17',
 'ACTOR_18',
 'ACTOR_19',
 'ACTOR_20',
 'ACTOR_21',
 'ACTOR_22',
 'ACTOR_23',
 'COMPOSER_0',
 'COMPOSER_1',
 'COMPOSER_2',
 'COMPOSER_3',
 'COMPOSER_4',
 'COMPOSER_5',
 'COMPOSER_6',
 'COMPOSER_7',
 'COMPOSER_8',
 'COMPOSER_9',
 'COMPOSER

In [166]:
with open('dataset_2kp.pkl', 'wb') as f:
    pickle.dump(df_pr, f)